## Trabalho Prático: Autovalores e SVD
### Algebra Linear Númerica
**Mírian e Erick** 

[Referências](http://people.inf.ethz.ch/arbenz/ewp/Lnotes/2010/chapter3.pdf)

In [441]:
using LinearAlgebra
using Printf
Base.show(io::IO, f::Float64) = @printf(io, "%1.3f", f)

In [442]:
#Matrizes de Exemplo:
#Exemplo A
ExA = [6.0 5.0 2.0 3.0; 
        5.0 1.0 4.0 6.0; 
        1.0 4.0 3.0 7.0;
        4.0 1.0 2.0 2.0]

4×4 Array{Float64,2}:
 6.000  5.000  2.000  3.000
 5.000  1.000  4.000  6.000
 1.000  4.000  3.000  7.000
 4.000  1.000  2.000  2.000

In [443]:
#Exemplo B
ExB = [6.0 5.0 0.0;
        5.0 1.0 4.0;
        0.0 4.0 3.0]

3×3 Array{Float64,2}:
 6.000  5.000  0.000
 5.000  1.000  4.000
 0.000  4.000  3.000

## _Primitivas_

### Questão (1)
Implementação de uma função que calcula a norma de um vetor no $R^n$.

In [520]:
#Norma Comum
function NormaComumdoVetor(V)
    sumV=0
    for i in V #Somatório do Elemento
        sumV = sumV + (i)^2
    end
    NormaV = sqrt(sumV)

    return NormaV
end

#Norma W - Recebe um vetor V e retorna a versão vetor W definida na questão
function NormaWdoVetor(V)
    Beta =  maximum(V) #Beta é o maior valor do vetor V
    W = (1/Beta)*V # o vetor W é o produto entre inversa de Beta e vetor V
    sumW=0
    for i in W #Somatório do Elemento
        sumW = sumW + (i)^2
    end
    NormaW = sqrt(sumW)

    return NormaW
end

NormaWdoVetor (generic function with 1 method)

### Questão (2) Rotações de Givens - Caso Real

Implementação de uma função que recebe dois números reais, f e g, e calcula *c,s* e *r* com as propriedades de norma implementadas acima, sem usar funções trigonométricas na resposta final.

In [522]:
#Função Rotação de Givens, recebe a e b, e retorna r, c, s
function RotacaoDeGivens2x2(f,g)

    r = sqrt((f*f)+(g*g)) #Achando o valor de r
    c = f/r #Fórmula do valor de c
    s = -g/r #Fórmula do valor de s

    return r,c,s
end

RotacaoDeGivens2x2 (generic function with 1 method)

### Questão (3) Refletores de Householder - Caso Real

Função que recebe um vetor w e computa γ e v com as propriedades implementadas acima.

In [523]:
#Função Reflexão de Householder, recebe vetor W e retorna 
function ReflexaoDeHouseholder(W)
    
    SizeW = length(W) #Pegando o tamanho do Vetor W
    NormadeW = NormaComumdoVetor(W) #Já tirando a norma de W
    
    e = zeros(SizeW, 1) #Aqui criaremos o vetor elementar e do mesmo tamanho do W
    e[1] = 1 #O primeiro valor de e é 1
    
    α = NormadeW #α é a norma de W

    V = W - α*e
    
    NormadeV = NormaComumdoVetor(V) #Norma do V encontrado para acharmos o γ
    γ = 2/(NormadeV*NormadeV)
    
    return γ, V
end

ReflexaoDeHouseholder (generic function with 1 method)

In [524]:
#TESTE
γ, V = ReflexaoDeHouseholder([3; 4])
P = I - γ*V*(V')
P*[3; 4]
V

2×1 Array{Float64,2}:
 -2.000
  4.000

### Questão (4) Decomposição QR

Implementação de uma função que calcula a decomposição QR de uma matriz A:

In [525]:
#QR por Givens
function QR2x2porRotacaoDeGivens(A) 
    V = A[:,1]
    a = A[1]
    b = A[2]
    r,c,s = RotacaoDeGivens2x2Givens(a,b)
    Q = [c s ; -s c]
    return Q
end

#QR por Reflexão de Householder
function QRporReflexaoDeHouseholder(A) 
    Q=one(A)
    mA = size(A,1)
    nA = size(A,2)
    for i in 1:(nA-1)
        vtemp = A[i:mA,i] #vetor temporario que é enviado pra Reflexao
        γ, v = ReflexaoDeHouseholder(vtemp)
        vt = transpose(v) 
        P = I - γ*v*vt #formula de Householder
        Qi = P
        Q[:,i:nA] = Q[:,i:nA]*Qi
        A[i:mA, i:nA]=P*A[i:mA, i:nA] #Problema aqui em alocar os valores das submatrizes, nao tenho ideia de como fazer em julia por hora
    end
    return Q,A
end

QRporReflexaoDeHouseholder (generic function with 1 method)

In [526]:
#TESTE QR PRONTO
function QRpronto(A) 
    Q,R = qr(A)
end

QRpronto(ExA)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
4×4 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.679   0.385   0.583  -0.223
 -0.566  -0.400  -0.566  -0.446
 -0.113   0.785  -0.582   0.179
 -0.453  -0.274  -0.022   0.848
R factor:
4×4 Array{Float64,2}:
 -8.832  -4.869  -4.869  -7.133
  0.000   4.393   0.978   3.704
  0.000   0.000  -2.888  -5.765
  0.000   0.000   0.000  -0.402

In [527]:
QRpronto(ExB)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -1.000  0.000  0.000
  0.000  1.000  0.000
  0.000  0.000  1.000
R factor:
3×3 Array{Float64,2}:
 -7.810  -4.481  -2.561
  0.000   4.682   0.966
  0.000   0.000  -4.184

In [531]:
#TESTE
Q, R = QRporReflexaoDeHouseholder(ExB)
print("\nMatriz Triangular R:")
Q


Matriz Triangular R:

3×3 Array{Float64,2}:
 1.000  0.000  0.000
 0.000  1.000  0.000
 0.000  0.000  1.000

In [532]:
Q*R

3×3 Array{Float64,2}:
  7.810  4.481   2.561
 -0.000  4.682   0.966
  0.000  0.000  -4.184

In [533]:
Q'*Q

3×3 Array{Float64,2}:
 1.000  0.000  0.000
 0.000  1.000  0.000
 0.000  0.000  1.000

## _Autovalores e autovetores_

### Questão (5) Redução à forma de Hessenberg

Implementação de uma função que reduza uma matriz A a forma de Hessenberg:

In [623]:
function ReducaoAHessenberg(A)
    H=A
    Q=one(A)
    mA = size(A,1)
    nA = size(A,2)
    Qf = A
    for i in 1:(nA-2)
        j=0
        vtemp = H[i+1:mA,i]
        γ, v = ReflexaoDeHouseholder(vtemp)
        P = I - γ*v*v'
        Qi = P
        
        dimQi = size(Qi,1)
        TamanhoIdentidadeIterativa = zeros(mA-dimQi,nA-dimQi)
        Quni = one(TamanhoIdentidadeIterativa)
        
        Z = zeros(mA-i,i)
        Q =[Quni Z'; Z Qi]

        H=Q*H*Q'
    end
    return Q, H
end

# OBS: Eu não sei pq do nada alguns valores saem negativos. Talvez seja algo lá em cima, na hora de calcular a reflexão. 
# Mas os valores pelo menos batem com os das funções prontas de Julia, oq é menos mal. Mas o sinal está avacalhando! Encontre o erro Erick, não achei! 

matrixQ, matrixH = ReducaoAHessenberg(ExA)
matrixH

4×4 Array{Float64,2}:
  6.000  6.018   1.215  -0.555
  6.481  6.571   2.933   4.830
  0.000  7.073   0.811   2.870
 -0.000  0.000  -0.216  -1.382

In [624]:
#TESTE HESSENBERG PRONTO PRA CONFERIR COM O IMPLEMENTADO
F = hessenberg(ExA)
F.Q*F.H*F.Q'

4×4 Array{Float64,2}:
 6.000  5.000  2.000  3.000
 5.000  1.000  4.000  6.000
 1.000  4.000  3.000  7.000
 4.000  1.000  2.000  2.000

In [625]:
F.H

4×4 Array{Float64,2}:
  6.000  -6.018  -1.215  -0.555
 -6.481   6.571   2.933  -4.830
  0.000   7.073   0.811  -2.870
  0.000   0.000   0.216  -1.382

In [626]:
F.Q

4×4 LinearAlgebra.HessenbergQ{Float64,Array{Float64,2}}:
 1.000   0.000   0.000   0.000
 0.000  -0.772  -0.003  -0.636
 0.000  -0.154  -0.969   0.192
 0.000  -0.617   0.246   0.747

### Questão (6) Iteração de Francis de grau 2 

Entrada:

- H Matriz n × n Hessenberg propria.
- Q Matriz n × n tal que A = QHQ'

In [627]:
#Iteração de Francis
function IteraçãodeFrancis(H,Q) 
    A = Q*H*Q'
    R = H*Q'
    
    n = size(A,1)
    e1 = zeros(n-1)
    e1[1] = 1
    
    # Use the eigenvalues of the lower-right-hand 2x2 submatrix as shifts. Iterate until
    # you have isolated a pair of eigenvalues
    
    #shift = eigvals(A[n-1:n,n-1:n])
    p1, p2 = eigvals(A[n-1:n,n-1:n]) #autovalores da matrix 2x2 que serão os shifts 1 e 2
    pH = (A-p1*I)*(A-p2*I) #p1 e p2 são shifts a escolher 
    #Hchapeu = Tô perdida em saber quem realmente tem que ser esse cara
    #return Hchapeu
end

IteraçãodeFrancis (generic function with 1 method)

In [628]:
# auto-valores da submatriz 2x2
function WilkinsonShift( a, b, c )
    # Calcula shift de Wilkinson para matrizes simetricas 
    δ = (a-c)/2
    return c - ((sign(δ)*b^2)/(abs(δ) + sqrt(δ^2+b^2)))
end

n = size(ExA,1)
WilkinsonShift(ExA[n-1,n-1], ExA[n,n], ExA[n-1,n])

7.828

In [629]:
Q, H = ReducaoAHessenberg(ExA)
A = Q*H*Q'
n = size(A,1)
WilkinsonShift(A[n-1,n-1], A[n,n], A[n-1,n])

-3.348

In [630]:
Q, H = ReducaoAHessenberg(ExA)
IteraçãodeFrancis(H, Q)

4×4 Array{Float64,2}:
  77.929   87.688  18.919  -35.183
  85.175  106.184  21.532  -38.880
  44.525   55.007  11.681  -37.027
 -10.900  -11.892  -2.859    9.064

In [631]:
#Teste
W = [7.0000 7.2761 5.8120  -0.1397  9.0152   7.9363;
    12.3693 4.1307 18.9685 -1.2071  10.6833  2.4160;
    0      -7.1603 2.4478  -0.5656  -4.1814  -3.2510;
    0       0     -8.5988  2.9151  -3.4169   5.7230;
    0       0     0        1.0464  -2.8351   -10.9792;
    0       0     0        0       1.4143    5.3415]

n = size(W,1)
W[n-1:n,n-1:n]
eigvals(W[n-1:n,n-1:n])

2-element Array{Float64,1}:
 0.164
 2.342

### Questão (7) Shift de Rayleigh Generalizado

Implemente uma função que calcula os shifts que usaremos: 
O shift de Rayleigh generalizado (dois autovalores da matriz $H_{canto}$) se eles não forem reais, ou duas cópias do shift de Wilkinson (autovalores da matriz $H_{canto}$ mais proximo de $h_{n,n}$) caso contrário

In [632]:
using LinearAlgebra

#Shift de Raileigh para achar os autovalores p1 e p2 da matriz Hcanto
function ShiftDeRaileigh(H) 
    #Os shifts ρ1 e ρ2 serao os dois autovalores da matriz Hcanto se eles nao forem reais
    p1, p2 = eigvals(H[n-1:n,n-1:n]) #autovalores da matrix 2x2 que serão os shifts 1 e 2
    return p1, p2
end

ShiftDeRaileigh (generic function with 1 method)

### Questão (8) Aplicando $Q_0$ tal que $Q_0^1(p(H)e_1) = βe_1$

Encontre manualmente uma formula explícita para o vetor $(H − ρ_1I)(H − ρ_2I)e_1$ usando que H e Hessenberg. Use sua função da seção 1.3 para encontrar um refletor $Q_0$ pequeno (que opera em poucas linhas/colunas) com a propriedade desejada.

In [633]:
using LinearAlgebra

function EncontreQ0(H) 
    
    return Q0
end

EncontreQ0 (generic function with 1 method)

### Questão Extra - Voltando a forma de Hessenberg
Se A é simétrica, H é tridiagonal, pois uma matriz similar a uma simétrica é também simétrica. Se não quisermos calcular Q, implemente uma otimização que faça uma iteração de Francis (no caso simétrico) em tempo O(n) e verifique que sua função se comporta igual ao caso geral.
Dica: Aplicar os refletores do algoritmo numa matriz tridiagonal demora tempo O(1), pois, para um dado refletor, quase todas as linhas/colunas conterão zeros nas posicões relevantes.

In [634]:
using LinearAlgebra

function FrancisOdeN(A,H) 
    numColH = size(H,2)
    for i in 1:numColH
        Wtemp = H[:,i]
        γ, v = ReflexaoDeHouseholder(Wtemp)
        Qi = I - γ*v*vt #Falta consertar a parte de guardar os valores do Qi
    end
    return Q0
end

FrancisOdeN (generic function with 1 method)

### Questão (9) Juntando tudo no caso simétrico
Implemente o procedimento acima para fazer varias iterações de Francis. Faça seu código imprimir uma mensagem dizendo qual dos dois casos (Caso 1 ou Caso 2) reduziu o problema. Qual caso acontece mais frequentemente?

In [635]:
println("Hello World")

Hello World


### Questão Extra - Juntando tudo no caso simétrico
Para todo $k = 1, . . . , n$, verifique numericamente que $p(H)e_k$ pode ser expresso como combinação linear das colunas ${q1, . . . , qk}$, e portanto que estamos iterando subespaços. Lembre-se de se restringir ao pedaço da matriz que e Hessenberg próprio.

In [636]:
println("Hello World")

Hello World


## _SVD de matrizes reais $n \times m$ com $n \geqslant m$_

### Questão (10) Bidiagonalização de Golub–Kahan

Implemente o algoritmo de bidiagonalização de Golub–Kahan.

In [637]:
println("Hello World")

Hello World


### Questão (11) Golub–Reinsch: SVD de B bidiagonal própria

Implemente uma iteração do algoritmo de Golub–Reinsch

In [638]:
println("Hello World")

Hello World
